In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from itertools import product
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import os

# Load the dataset from two sheets
file_path = 'sam9.xlsx'  # Replace with your file path
sheet1 = pd.read_excel(file_path, sheet_name='Sheet_1')
sheet2 = pd.read_excel(file_path, sheet_name='Sheet_2')

# Concatenate data from both sheets
df = pd.concat([sheet1, sheet2], ignore_index=True)

# Preprocess the data
df['datetime'] = pd.to_datetime(df['order_date'].astype(str) + ' ' + df['order_time'].astype(str))
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df = df.dropna(subset=['quantity'])

# Ensure the data contains 10 years
min_date = df['datetime'].min()
max_date = df['datetime'].max()
total_days = (max_date - min_date).days

if total_days < 365 * 10:
    raise ValueError("The dataset does not contain 10 years of data.")

# Split the data into training (first 9 years) and validation (10th year)
cutoff_date = pd.to_datetime('2019-12-31')
train_df = df[df['datetime'] <= cutoff_date]
test_df = df[df['datetime'] > cutoff_date]

# Hyperparameter tuning grid (simplified)
param_grid = {
    'seasonality_mode': ['additive', 'multiplicative'],
    'changepoint_prior_scale': [0.1, 0.5],
    'seasonality_prior_scale': [1.0, 10.0]
}

def tune_prophet_model(train_data):
    best_model = None
    best_mape = float('inf')
    best_params = None

    for params in product(param_grid['seasonality_mode'], param_grid['changepoint_prior_scale'], param_grid['seasonality_prior_scale']):
        seasonality_mode, changepoint_prior_scale, seasonality_prior_scale = params
        
        model = Prophet(
            seasonality_mode=seasonality_mode,
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=True
        )
        
        model.add_seasonality(name='quarterly', period=90, fourier_order=5)
        
        try:
            model.fit(train_data)
        except Exception as e:
            print(f"Error fitting model with params {params}: {e}")
            continue
        
        future = model.make_future_dataframe(periods=365, freq='D', include_history=True)
        forecast = model.predict(future)
        
        forecast_train = forecast[forecast['ds'] <= cutoff_date]
        merged = pd.merge(train_data, forecast_train[['ds', 'yhat']], on='ds', how='left')
        
        mape = np.mean(np.abs((merged['y'] - merged['yhat']) / merged['y'])) * 100

        if mape < best_mape:
            best_mape = mape
            best_model = model
            best_params = params

    return best_model, best_params

# Function to forecast and evaluate for a given pizza
def forecast_pizza(pizza_name, pizza_ingredients, group):
    train_agg = group.groupby(pd.Grouper(key='datetime', freq='D')).agg({'quantity': 'sum'}).reset_index()
    train_prophet = train_agg[['datetime', 'quantity']]
    train_prophet.columns = ['ds', 'y']

    best_model, best_params = tune_prophet_model(train_prophet)
    if best_model is None:
        best_model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)
        best_model.add_seasonality(name='quarterly', period=90, fourier_order=5)
        best_model.fit(train_prophet)

    future = best_model.make_future_dataframe(periods=366, freq='D')
    forecast = best_model.predict(future)
    forecast_5th_year = forecast[forecast['ds'] > cutoff_date]

    test_group = test_df[(test_df['pizza_name'] == pizza_name) & (test_df['pizza_ingredients'] == pizza_ingredients)]
    test_agg = test_group.groupby(pd.Grouper(key='datetime', freq='D')).agg({'quantity': 'sum'}).reset_index()
    test_agg.columns = ['ds', 'y']

    comparison_df = pd.merge(test_agg, forecast_5th_year[['ds', 'yhat']], on='ds', how='left')
    comparison_df = comparison_df.dropna(subset=['y', 'yhat'])
    comparison_df = comparison_df[comparison_df['y'] != 0]

    mae = mean_absolute_error(comparison_df['y'], comparison_df['yhat'])
    mse = mean_squared_error(comparison_df['y'], comparison_df['yhat'])
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((comparison_df['y'] - comparison_df['yhat']) / comparison_df['y'])) * 100
    accuracy = 100 - mape

    actual_sum = comparison_df['y'].sum()
    predicted_sum = comparison_df['yhat'].sum()

    return {
        'pizza_name': pizza_name,
        'pizza_ingredients': pizza_ingredients,
        'comparison_df': comparison_df,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'Accuracy': accuracy,
        'Actual_Sum': actual_sum,
        'Predicted_Sum': predicted_sum
    }

# Use parallel processing to forecast for each pizza
results = Parallel(n_jobs=-1)(delayed(forecast_pizza)(pizza_name, pizza_ingredients, group) 
                              for (pizza_name, pizza_ingredients), group in train_df.groupby(['pizza_name', 'pizza_ingredients']))

# Convert results to a DataFrame
metrics_df = pd.DataFrame([{
    'pizza_name': res['pizza_name'],
    'pizza_ingredients': res['pizza_ingredients'],
    'MAE': res['MAE'],
    'MSE': res['MSE'],
    'RMSE': res['RMSE'],
    'MAPE': res['MAPE'],
    'Accuracy': res['Accuracy'],
    'Actual_Sum': res['Actual_Sum'],
    'Predicted_Sum': res['Predicted_Sum']
} for res in results])

# Write metrics and forecasts to an Excel file with plots
output_excel_path = 'forecast_results.xlsx'
with pd.ExcelWriter(output_excel_path, engine='xlsxwriter') as writer:
    # Write metrics to the first sheet
    metrics_df.to_excel(writer, sheet_name='Metrics', index=False)
    
    workbook = writer.book
    worksheet = writer.sheets['Metrics']
    
    # Create a bar chart for Actual and Predicted sums
    chart = workbook.add_chart({'type': 'column'})
    
    # Configure the first series for Actual Sum
    chart.add_series({
        'name': 'Actual Sum',
        'categories': ['Metrics', 1, 0, len(metrics_df), 0],
        'values': ['Metrics', 1, 7, len(metrics_df), 7],
    })
    
    # Configure the second series for Predicted Sum
    chart.add_series({
        'name': 'Predicted Sum',
        'categories': ['Metrics', 1, 0, len(metrics_df), 0],
        'values': ['Metrics', 1, 8, len(metrics_df), 8],
    })
    
    # Add chart title and axes labels
    chart.set_title({'name': 'Prophet Actual vs Predicted '})
    chart.set_x_axis({'name': 'Pizza Type'})
    chart.set_y_axis({'name': 'Total Quantities'})
    
    # Insert the chart into the worksheet
    worksheet.insert_chart('K2', chart)
    
    for res in results:
        pizza_name = res['pizza_name'][:15].replace('/', '-')
        pizza_ingredients = res['pizza_ingredients'][:15].replace('/', '-')
        sheet_name = f"{pizza_name}_{pizza_ingredients}"
        
        # Save comparison dataframe to a sheet
        res['comparison_df'].to_excel(writer, sheet_name=sheet_name[:31], index=False)
        
        # Plot the results
        plt.figure(figsize=(10, 6))
        plt.plot(res['comparison_df']['ds'], res['comparison_df']['y'], label='Actual Quantity')
        plt.plot(res['comparison_df']['ds'], res['comparison_df']['yhat'], label='Forecasted Quantity')
        plt.xlabel('Date')
        plt.ylabel('Quantity')
        plt.title(f"Forecast vs Actual for {pizza_name} ({pizza_ingredients})")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        
        # Save the plot
        plot_path = f"{pizza_name}_{pizza_ingredients}.png"
        plt.savefig(plot_path)
        plt.close()
        
        # Insert the plot into the Excel file
        worksheet = writer.sheets[sheet_name[:31]]
        worksheet.insert_image('G2', plot_path)

# Remove temporary plot files
for res in results:
    pizza_name = res['pizza_name'][:15].replace('/', '-')
    pizza_ingredients = res['pizza_ingredients'][:15].replace('/', '-')
    plot_path = f"{pizza_name}_{pizza_ingredients}.png"
    if os.path.exists(plot_path):
        os.remove(plot_path)

# Print the metrics for each pizza name and ingredients
print(metrics_df)

# Overall metrics across all pizzas
overall_mae = metrics_df['MAE'].mean()
overall_mse = metrics_df['MSE'].mean()
overall_rmse = metrics_df['RMSE'].mean()
overall_mape = metrics_df['MAPE'].mean()
overall_accuracy = 100 - overall_mape

print(f"Overall Mean Absolute Error (MAE): {overall_mae}")
print(f"Overall Mean Squared Error (MSE): {overall_mse}")
print(f"Overall Root Mean Squared Error (RMSE): {overall_rmse}")
print(f"Overall Mean Absolute Percentage Error (MAPE): {overall_mape}%")
print(f"Overall Accuracy: {overall_accuracy}%")
